In [8]:
# 1- Importar el archivo csv y extraer las variables Total_Score, Study_hours_per_Week y Attendance(%)
# 2- Limpiar datos aplicando al menos un algoritmo de deteccion de outliers
# 3- Normalizar/estandarizar datos
# 4- Agrupamiento K-means
# 5- Etiquetado y descripcion de clusters 

In [ ]:
import pandas as pd
datos = pd.read_csv("datos.csv")

datosR= datos[['Total_Score','Study_Hours_per_Week', 'Attendance (%)' ]]

In [ ]:
#Normalizacion de los datos:
#Si no tienen sklearn, Utilicen el comando
#pip install scikit-learn

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalización (MinMaxScaler)
scaler_minmax = MinMaxScaler()
datos_normalizados = scaler_minmax.fit_transform(datosR)

# Convertir el resultado de la normalización a DataFrame
datos_normalizados_df = pd.DataFrame(datos_normalizados, columns=['Total_Score', 'Study_Hours_per_Week', 'Attendance (%)'])

# Estandarización (StandardScaler)
scaler_standard = StandardScaler()
datos_estandarizados = scaler_standard.fit_transform(datosR)

# Convertir el resultado de la estandarización a DataFrame
datos_estandarizados_df = pd.DataFrame(datos_estandarizados, columns=['Total_Score', 'Study_Hours_per_Week', 'Attendance (%)'])

# Mostrar resultados (Los primeros 4 datos)
print("Datos Normalizados:")
print(datos_normalizados_df.head())

print("\nDatos Estandarizados:")
print(datos_estandarizados_df.head())

#IMPORTANTEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEoeoeoeoeoepotoeoeoeo
#Solo necesitamos los datos estandarizados O normalizados
#Se debe elegir si utilizar los datos normalizados O estandarizados, y porque 

Datos Normalizados:
   Total_Score  Study_Hours_per_Week  Attendance (%)
0     0.121473                 0.048        0.045609
1     0.012407                 0.560        0.945389
2     0.405844                 0.628        0.143629
3     0.232339                 0.792        0.902981
4     0.322393                 0.416        0.083417

Datos Estandarizados:
   Total_Score  Study_Hours_per_Week  Attendance (%)
0    -1.321791             -1.575071       -1.610303
1    -1.700302              0.184346        1.519646
2    -0.334882              0.418018       -1.269335
3    -0.937028              0.981582        1.372125
4    -0.624496             -0.310490       -1.478787


AttributeError: 'numpy.ndarray' object has no attribute 'describe'